In [1]:
import scrapy
from scrapy import Selector
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [17]:
import nltk

In [2]:
#initialize two empty lists
latlib_data = []

latlib_titles = []



In [3]:
#initialize an empty DataFrame
latlib_df = pd.DataFrame([])


In [4]:
class LatLibSpider(scrapy.Spider):
    name = 'latlib_spider'
    def start_requests(self):
        #This starts the process. Here the base url is included.
        
        urls = 'https://www.thelatinlibrary.com/'
        yield scrapy.Request( url = urls, callback = self.parse_front)
    def parse_front(self, response):
        #opening latinlibrary page
        #parser to get urls for each author's individual works
        
        author_links = response.xpath('.//table[2]/tr/td/a/@href').extract() 
        head_url = 'https://www.thelatinlibrary.com/'
        author_links = [head_url+l for l in author_links] 
        for link in author_links:
            yield response.follow(url = link, callback = self.parse_authors)
            
    def parse_authors(self, response):
        #navigate to each author page
        #This is to extract individual author links. 
        #This includes the xpath location of each author's html
        #then appends it onto the base url using list comprehension.
        
        author_page = response.xpath('//div[@class="work"]/table/tbody/tr/td/a/@href').extract() 
        for link in author_page:
            yield response.follow(url = link, callback = self.parse_works)

    def parse_works(self, response):
        #get the title of each work:
        #This parser is used to scrape the actual text and title from each work
        #as well as append that data to the empty lists at the top of the file.
        
        title_text = response.xpath('/html/head/title/text()').extract()
        #get the content of each work under the authors
        body_text = response.xpath('.//body/*/text()').extract() 

        #latlib_data = latlib_data.append(title_text)
        latlib_data.append(body_text) 
        latlib_titles.append(title_text) 
        
        


In [5]:
#This is where I actually run the Spider
process = CrawlerProcess()
process.crawl(LatLibSpider)
process.start()

2021-03-22 11:52:58 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-03-22 11:52:58 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.5 (default, Sep  4 2020, 02:22:02) - [Clang 10.0.0 ], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform macOS-10.15.5-x86_64-i386-64bit
2021-03-22 11:52:58 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-03-22 11:52:58 [scrapy.crawler] INFO: Overridden settings:
{}
2021-03-22 11:52:58 [scrapy.extensions.telnet] INFO: Telnet Password: a570fad4648ad1dc
2021-03-22 11:52:58 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2021-03-22 11:52:58 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddle

2021-03-22 11:52:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/sen/sen.vita.shtml> (referer: https://www.thelatinlibrary.com/sen.html)
2021-03-22 11:52:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/sen/sen.medea.shtml> (referer: https://www.thelatinlibrary.com/sen.html)
2021-03-22 11:52:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/sen/sen.clem.shtml> (referer: https://www.thelatinlibrary.com/sen.html)
2021-03-22 11:52:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/sall.ep1.html> (referer: https://www.thelatinlibrary.com/sall.html)
2021-03-22 11:52:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/sall.mithr.html> (referer: https://www.thelatinlibrary.com/sall.html)
2021-03-22 11:52:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/sall.invectiva.html> (referer: https://www.thelatin

2021-03-22 11:53:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/quintilian/quintilian.decl.mai10.shtml> (referer: https://www.thelatinlibrary.com/quintilian.html)
2021-03-22 11:53:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/quintilian/quintilian.decl.mai4.shtml> (referer: https://www.thelatinlibrary.com/quintilian.html)
2021-03-22 11:53:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/quintilian/quintilian.decl.mai8.shtml> (referer: https://www.thelatinlibrary.com/quintilian.html)
2021-03-22 11:53:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/quintilian/quintilian.decl.mai7.shtml> (referer: https://www.thelatinlibrary.com/quintilian.html)
2021-03-22 11:53:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/quintilian/quintilian.decl.mai6.shtml> (referer: https://www.thelatinlibrary.com/quintilian.html)
2021-03-22 11:

2021-03-22 11:53:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/sen/sen.qn6.shtml> (referer: https://www.thelatinlibrary.com/sen.html)
2021-03-22 11:53:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/sen/sen.qn4.shtml> (referer: https://www.thelatinlibrary.com/sen.html)
2021-03-22 11:53:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/sen/sen.qn2.shtml> (referer: https://www.thelatinlibrary.com/sen.html)
2021-03-22 11:53:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/sen/seneca.ep20.shtml> (referer: https://www.thelatinlibrary.com/sen.html)
2021-03-22 11:53:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/sen/seneca.ep17-18.shtml> (referer: https://www.thelatinlibrary.com/sen.html)
2021-03-22 11:53:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/sen/seneca.ep19.shtml> (referer: https:/

2021-03-22 11:53:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/suetonius/suet.otho.html> (referer: https://www.thelatinlibrary.com/suet.html)
2021-03-22 11:53:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/suetonius/suet.claudius.html> (referer: https://www.thelatinlibrary.com/suet.html)
2021-03-22 11:53:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/suetonius/suet.cal.html> (referer: https://www.thelatinlibrary.com/suet.html)
2021-03-22 11:53:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/suetonius/suet.nero.html> (referer: https://www.thelatinlibrary.com/suet.html)
2021-03-22 11:53:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/suetonius/suet.caesar.html> (referer: https://www.thelatinlibrary.com/suet.html)
2021-03-22 11:53:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/sueton

...

2021-03-22 11:53:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ovid/ovid.her4.shtml> (referer: https://www.thelatinlibrary.com/ovid.html)
2021-03-22 11:53:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ovid/ovid.her2.shtml> (referer: https://www.thelatinlibrary.com/ovid.html)
2021-03-22 11:53:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ovid/ovid.amor2.shtml> (referer: https://www.thelatinlibrary.com/ovid.html)
2021-03-22 11:53:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ovid/ovid.amor3.shtml> (referer: https://www.thelatinlibrary.com/ovid.html)
2021-03-22 11:53:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ovid/ovid.amor1.shtml> (referer: https://www.thelatinlibrary.com/ovid.html)
2021-03-22 11:53:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ovid/ovid.met14.shtml> (refe

2021-03-22 11:53:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/pliny.nh1.html> (referer: https://www.thelatinlibrary.com/pliny1.html)
2021-03-22 11:53:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/vergil/ec6.shtml> (referer: https://www.thelatinlibrary.com/verg.html)
2021-03-22 11:53:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/vergil/ec5.shtml> (referer: https://www.thelatinlibrary.com/verg.html)
2021-03-22 11:53:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/vergil/ec4.shtml> (referer: https://www.thelatinlibrary.com/verg.html)
2021-03-22 11:53:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/vergil/ec3.shtml> (referer: https://www.thelatinlibrary.com/verg.html)
2021-03-22 11:53:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/vergil/ec2.shtml> (referer: https://www.thelatinlib

2021-03-22 11:53:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/eutropius/eutropius10.shtml> (referer: https://www.thelatinlibrary.com/eutropius.html)
2021-03-22 11:53:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/eutropius/eutropius3.shtml> (referer: https://www.thelatinlibrary.com/eutropius.html)
2021-03-22 11:53:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/eutropius/eutropius4.shtml> (referer: https://www.thelatinlibrary.com/eutropius.html)
2021-03-22 11:53:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/eutropius/eutropius6.shtml> (referer: https://www.thelatinlibrary.com/eutropius.html)
2021-03-22 11:53:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/eutropius/eutropius2.shtml> (referer: https://www.thelatinlibrary.com/eutropius.html)
2021-03-22 11:53:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https:/

2021-03-22 11:53:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ammianus/22.shtml> (referer: https://www.thelatinlibrary.com/ammianus.html)
2021-03-22 11:53:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ammianus/19.shtml> (referer: https://www.thelatinlibrary.com/ammianus.html)
2021-03-22 11:53:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ammianus/18.shtml> (referer: https://www.thelatinlibrary.com/ammianus.html)
2021-03-22 11:53:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ammianus/17.shtml> (referer: https://www.thelatinlibrary.com/ammianus.html)
2021-03-22 11:53:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ammianus/15.shtml> (referer: https://www.thelatinlibrary.com/ammianus.html)
2021-03-22 11:53:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/ammianus/16.shtml> (refere

2021-03-22 11:53:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/cicero/postreditum2.shtml> (referer: https://www.thelatinlibrary.com/cic.html)
2021-03-22 11:53:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/cicero/postreditum.shtml> (referer: https://www.thelatinlibrary.com/cic.html)
2021-03-22 11:53:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/cicero/arch.shtml> (referer: https://www.thelatinlibrary.com/cic.html)
2021-03-22 11:53:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/cicero/plancio.shtml> (referer: https://www.thelatinlibrary.com/cic.html)
2021-03-22 11:53:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/cicero/sulla.shtml> (referer: https://www.thelatinlibrary.com/cic.html)
2021-03-22 11:53:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.thelatinlibrary.com/cicero/flacco.shtml> (referer: h

In [6]:
ts = ''
for i in latlib_titles:
    istr = ''.join(i).strip('\n').strip('\t').strip('\xa0')
    #print(istr)
    ts+=istr

print(ts)

Varro: Lingua Latina, Fragmenta
Varro: Lingua Latina X
Varro: Lingua Latina VIIII
Statius: Achilleid II
Varro: Lingua Latina VII
Varro: Lingua Latina VIII
Varro: Lingua Latina VI
Tacitus: Agricola
Tacitus: Histories V
Tacitus: Germania
Tacitus: Dialogus de Oratoribus
Terence: Adelphoe
Terence: Heauton Timorumenos
Terence: Phormio
Sallust: Letter to Caesar II
Terence: Eunuchus
Seneca: On the Good Life
Seneca: Medea
Seneca: On Clemency
Sallust: Letter to Caesar I
Sallust: Speech of Mithridates
Sallust: Invective Against Cicero
Sallust: Speech of Macer
Sallust: Speech of Pompey
Sallust: Speech of Cotta
Sallust: Speech of Philippus
Sallust: Speech of Lepidus
Bellum Catilinae
Seneca the Elder
Sallust: Bellum Iugurthinum
Seneca the Elder
Seneca the Elder
Seneca the Elder
Seneca the Elder
Seneca the Elder
Seneca the Elder
Seneca the Elder
Seneca the Elder
Pliny the Younger
Seneca the Elder
Pliny the Younger
Pliny the Younger
Pliny the Younger
Pliny the Younger
Pliny the Younger
Pliny the Youn

In [16]:
for d in latlib_data:
    dstr = ''.join(d)
    print(dstr.strip('\n')[:500])

M. TERENTI VARRONIS DE LINGUA LATINA

FRAGMENTA



1. Deus autem vel dea generale nomen est omnibus  * * *  Varro ad Ciceronem
tertio: "Ita respondeant cur dicant deos, cum de omnibus antiqui dixerint
divos."

2. Figor ambigue declinatur apud veteres tempore perfecto. Reperimus
enim fictus et fixus  * * *  Varro ad Ciceronem tertio "fixum".

3. Hoc nomen licet veteres Latinum negent, auctoritate tamen valet.
Dicebant enim leonem masculum et feminam  * * * . Leam vero Varro ad Ciceronem
dicit lib
M. TERENTI VARRONIS DE LINGUA LATINA

LIBER X


I. 

In verborum declinationibus disciplina loquendi dissimilitudinem an similitudinem sequi deberet, multi quaesierunt. Cum ab his ratio quae ab similitudine oriretur vocaretur analogia, reliqua pars appellaretur anomalia: de qua re primo libro quae dicerentur cur dissimilitudinem ducem haberi oporteret, dixi, secundo contra quae dicerentur, cur potius similitudinem conveniret praeponi: quarum rerum quod nec fundamenta, ut debuit, posita ab ullo 

In [8]:
#here's an example of a text and its title!
print(latlib_titles[10])
print(latlib_data[10])

['Tacitus: Dialogus de Oratoribus\n\t\t']
['P. CORNELI TACITI DIALOGVS DE ORATORIBVS\n', '\n', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \t', ' \n', '\n[', ']  Saepe ex me requiris, Iuste Fabi, cur, cum priora saecula tot eminentium oratorum ingeniis gloriaque floruerint,  nostra potissimum aetas deserta et laude eloquentiae orbata  vix nomen ipsum oratoris retineat; neque enim ita appellamus nisi antiquos, horum autem temporum diserti causidici et advocati et patroni et quidvis potius quam oratores vocantur. Cui percontationi tuae respondere et tam magnae quaestionis pondus excipere, ut aut de ingeniis nostris male  existimandum [sit], si idem adsequi non possumus, aut de  iudiciis, si nolumus, vix hercule auderem, si mihi mea sententia proferenda a

In [9]:
#number of texts:
len(latlib_data)

673

In [10]:
#length of the first text:
len(latlib_data[0])

64

In [11]:
#This is where I converted the lists containing the title and text data into Series
#and added them into a DataFrame.
latlib_df['title'] = pd.Series(latlib_titles)
latlib_df['text'] = pd.Series(latlib_data)

In [19]:
#toks = latlib_df.text.map(nltk.word_tokenize)
#latlib_df['tokens'] = toks

In [20]:
#latlib_df['token_count'] = latlib_df.tokens.map(len)

In [12]:
latlib_df.sample(10)

,title,text
650,[\n\t\t\tCicero: Speeches Against Catiline\n\t\t],"[M. TVLLI CICERONIS ORATIONES IN CATILINAM, \n..."
50,[\n\t\t\tPropertius Book III\n\t\t],"[SEXTI PROPERTI ELEGIARVM LIBER TERTIVS\n, \n,..."
413,[\n\t\t\tNepos: Life of Hamilcar\n\t\t],"[CORNELI NEPOTIS HAMILCAR\n\n, \n[1] HAMILCAR..."
441,[\n\t\t\tOvid: Ex Ponto I\n\t\t],"[P. OVIDI NASONIS EPISTVLAE EX PONTO, \nLIBER ..."
615,[\n\t\t\tCicero: de Legibus\n\t\t],"[M. TVLLI CICERONIS DE LEGIBVS LIBRI TRES\n, \..."
257,[ Florus:\nEpitome of Roman Wars ],"[\n\n, \n\n, \n\n, \n\n, \n\n, \n\n, \n\n, \n..."
205,[\n\t\t\tStatius: Silvae IV\n\t\t],"[PVBLIVS PAPINIVS STATIVS, \nSILVAE IV, \n, \n..."
161,[\n\t\t\tTiberius\n\t\t],"[SVETONI TRANQVILII VITA TIBERI\n, \n\n\t, \n\..."
480,[\n\t\t\tOvid: Metamorphoses V\n\t\t],[P. OVIDI NASONIS METAMORPHOSEON LIBER QVINTVS...
354,[\n\t\t\tLivy: Book XXXIV\n\t\t],"[TITI LIVI AB VRBE CONDITA LIBER XXXIV\n\n, \t..."


In [13]:
#Here is the pickle:
latlib_df.to_pickle("/Users/francesharrington/Documents/ling1340/Latin-Vocabulary-Analysis/data/latinlibrary_data.pkl")

In [14]:
#to make a set of sample data I'll take ten random files and pickle them:
samples = latlib_df.sample(10)

In [15]:
samples.to_csv("/Users/francesharrington/Documents/ling1340/Latin-Vocabulary-Analysis/data_samples/ten_samples.csv")